In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import gc
import xgboost as xgb

pd.set_option('display.max_columns', 200)

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Machine Learning Project - 605/XGBoost/train/train.csv', engine='python')
test_df = pd.read_csv('/content/drive/MyDrive/Machine Learning Project - 605/XGBoost/test/test.csv', engine='python')

In [ ]:
import subprocess
print((subprocess.check_output("lscpu", shell=True).strip()).decode())

Architecture:                         x86_64
CPU op-mode(s):                       32-bit, 64-bit
Address sizes:                        46 bits physical, 48 bits virtual
Byte Order:                           Little Endian
CPU(s):                               2
On-line CPU(s) list:                  0,1
Vendor ID:                            GenuineIntel
Model name:                           Intel(R) Xeon(R) CPU @ 2.20GHz
CPU family:                           6
Model:                                79
Thread(s) per core:                   2
Core(s) per socket:                   1
Socket(s):                            1
Stepping:                             0
BogoMIPS:                             4399.99
Flags:                                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 

In [ ]:
!nvidia-smi

Mon May  5 21:00:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
ITERATIONS = 700
EARLY_STOP = 10

params = {'tree_method': 'gpu_hist', 'max_depth': 8, 'alpha': 0.1,
          'gamma': 0.3, 'subsample': 0.6, 'scale_pos_weight': 1, 'learning_rate': 0.05,
          'silent': 1, 'objective':'binary:logistic', 'eval_metric': 'auc', 'silent':True,
          'verbose_eval': False}

In [ ]:
import time
nfold = 5
skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=2019)

oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))

target = 'target'
predictors = train_df.columns.values.tolist()[2:]

total_train_time = 0
total_pred_time = 0

i = 1
for train_index, valid_index in skf.split(train_df, train_df.target.values):
    print("\nFold {}".format(i))
    xg_train = xgb.DMatrix(train_df.iloc[train_index][predictors].values,
                           train_df.iloc[train_index][target].values,
                           )
    xg_valid = xgb.DMatrix(train_df.iloc[valid_index][predictors].values,
                           train_df.iloc[valid_index][target].values,
                           )

    # Start timing training
    start_train = time.time()
    clf = xgb.train(params, xg_train, ITERATIONS, evals=[(xg_train, "train"), (xg_valid, "eval")])
    end_train = time.time()
    total_train_time += (end_train - start_train)

    # Start timing prediction
    start_pred = time.time()
    oof[valid_index] = clf.predict(xgb.DMatrix(train_df.iloc[valid_index][predictors].values))
    predictions += clf.predict(xgb.DMatrix(test_df[predictors].values)) / nfold
    end_pred = time.time()
    total_pred_time += (end_pred - start_pred)

    i = i + 1

print(f"\nTotal training time: {total_train_time:.2f} seconds")
print(f"Total prediction time: {total_pred_time:.2f} seconds")


Fold 1


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:04:04] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:04:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent", "verbose_eval" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-auc:0.67114	eval-auc:0.64978
[1]	train-auc:0.70256	eval-auc:0.67687
[2]	train-auc:0.72201	eval-auc:0.68959
[3]	train-auc:0.73752	eval-auc:0.70003
[4]	train-auc:0.74759	eval-auc:0.70731
[5]	train-auc:0.75718	eval-auc:0.71186
[6]	train-auc:0.76385	eval-auc:0.71648
[7]	train-auc:0.77220	eval-auc:0.72025
[8]	train-auc:0.77775	eval-auc:0.72357
[9]	train-auc:0.78295	eval-auc:0.72736
[10]	train-auc:0.78899	eval-auc:0.73049
[11]	train-auc:0.79619	eval-auc:0.73396
[12]	train-auc:0.80211	eval-auc:0.73794
[13]	train-auc:0.80494	eval-auc:0.74001
[14]	train-auc:0.81223	eval-auc:0.74442
[15]	train-auc:0.81675	eval-auc:0.74794
[16]	train-auc:0.82388	eval-auc:0.75274
[17]	train-auc:0.82882	eval-auc:0.75526
[18]	train-auc:0.83038	eval-auc:0.75655
[19]	train-auc:0.83576	eval-auc:0.75949
[20]	train-auc:0.84091	eval-auc:0.76226
[21]	train-auc:0.84436	eval-auc:0.76459
[22]	train-auc:0.84718	eval-auc:0.76583
[23]	train-auc:0.85200	eval-auc:0.76897
[24]	train-auc:0.85504	eval-auc:0.77161
[25]	train

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:04:24] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)



Fold 2


...


Fold 5


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:05:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:05:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent", "verbose_eval" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-auc:0.67279	eval-auc:0.63613
[1]	train-auc:0.70514	eval-auc:0.66816
[2]	train-auc:0.71820	eval-auc:0.67880
[3]	train-auc:0.73374	eval-auc:0.69119
[4]	train-auc:0.74671	eval-auc:0.69750
[5]	train-auc:0.75582	eval-auc:0.70371
[6]	train-auc:0.76751	eval-auc:0.71063
[7]	train-auc:0.77731	eval-auc:0.71715
[8]	train-auc:0.78039	eval-auc:0.71884
[9]	train-auc:0.78581	eval-auc:0.72356
[10]	train-auc:0.79182	eval-auc:0.72723
[11]	train-auc:0.79829	eval-auc:0.73170
[12]	train-auc:0.80455	eval-auc:0.73431
[13]	train-auc:0.81052	eval-auc:0.73878
[14]	train-auc:0.81856	eval-auc:0.74248
[15]	train-auc:0.82159	eval-auc:0.74460
[16]	train-auc:0.82773	eval-auc:0.74918
[17]	train-auc:0.83281	eval-auc:0.75278
[18]	train-auc:0.83675	eval-auc:0.75640
[19]	train-auc:0.84194	eval-auc:0.75833
[20]	train-auc:0.84748	eval-auc:0.76165
[21]	train-auc:0.85065	eval-auc:0.76328
[22]	train-auc:0.85270	eval-auc:0.76477
[23]	train-auc:0.85545	eval-auc:0.76637
[24]	train-auc:0.85962	eval-auc:0.76924
[25]	train

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:05:57] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)



Total training time: 100.04 seconds
Total prediction time: 9.92 seconds


In [ ]:
import cupy as cp
def print_memory_usage_gb():
  device = cp.cuda.Device(0)
  free_mem, total_mem = device.mem_info
  free_mem /= (1024 ** 3)
  total_mem /= (1024 ** 3)
  used_mem = total_mem - free_mem
  print(f"GPU VRAM Usage: {used_mem:.4f} GB out of {total_mem:.4f} GB")

print_memory_usage_gb()

GPU VRAM Usage: 0.5068 GB out of 14.7413 GB
